1: Considere o seguinte cenário: Um vetor com 10000 posicoes que representam valores decimais "notas academicas".

Com bases nestes dados, é necessário construir um histograma, o intervalo deste histograma representa intervalos de 1 com limite de 10.

Exemplo: de 0 a 1, de 1.1 a 2, de 2.1 a 3, de 9.1 a 10.

Com base nessa situcação entregue as seguintes respostas:

- O valor de cada faixa do histograma
- A soma destas faixas
- O tempo de processamento

Faça para uma modelagem serial e paralela.

Código Serial em Python:




In [4]:
import numpy as np
import time

NUM_NOTAS = 10000000
NUM_FAIXAS = 10

notas = np.random.uniform(0, 10, NUM_NOTAS)

faixas = [(i * 1.0, (i + 1) * 1.0) for i in range(NUM_FAIXAS)]
faixas = [(faixa[0], faixa[1] if i < NUM_FAIXAS - 1 else 10) for i, faixa in enumerate(faixas)]

contagem_histograma = [0] * NUM_FAIXAS
soma_histograma = [0.0] * NUM_FAIXAS

start_time = time.time()

for nota in notas:
    for i in range(NUM_FAIXAS):
        if faixas[i][0] <= nota < faixas[i][1]:
            soma_histograma[i] += nota
            contagem_histograma[i] += 1
            break

end_time = time.time()
execution_time = end_time - start_time  # Calcula o tempo de execução

for i in range(NUM_FAIXAS):
    print(f"Faixa {faixas[i][0]:.1f} a {faixas[i][1]:.1f}: Soma = {soma_histograma[i]:.2f}, Contagem = {contagem_histograma[i]}")

print(f"\nTempo de execução: {execution_time:.6f} segundos")


Faixa 0.0 a 1.0: Soma = 499882.55, Contagem = 1000384
Faixa 1.0 a 2.0: Soma = 1501629.17, Contagem = 1000833
Faixa 2.0 a 3.0: Soma = 2499157.62, Contagem = 999782
Faixa 3.0 a 4.0: Soma = 3493081.28, Contagem = 998003
Faixa 4.0 a 5.0: Soma = 4495425.53, Contagem = 998922
Faixa 5.0 a 6.0: Soma = 5500863.48, Contagem = 1000112
Faixa 6.0 a 7.0: Soma = 6510598.50, Contagem = 1001656
Faixa 7.0 a 8.0: Soma = 7496786.51, Contagem = 999591
Faixa 8.0 a 9.0: Soma = 8504827.06, Contagem = 1000530
Faixa 9.0 a 10.0: Soma = 9501637.96, Contagem = 1000187

Tempo de execução: 22.476538 segundos


Código Paralelisado em CUDA:



In [13]:
%%writefile histograma_cuda.cu
#include <stdio.h>
#include <stdlib.h>

#define THREADS_PER_BLOCK 256

__global__ void calcular_histograma(float *notas, float *faixas, int num_notas, int num_faixas, float *soma_histograma, int *contagem_histograma) {
    int idx = blockDim.x * blockIdx.x + threadIdx.x;  // índice global da thread
    if (idx < num_notas) {
        float nota = notas[idx];
        for (int i = 0; i < num_faixas; i++) {
            if (nota >= faixas[2 * i] && nota <= faixas[2 * i + 1]) {
                atomicAdd(&soma_histograma[i], nota);  // Soma atômica
                atomicAdd(&contagem_histograma[i], 1); // Contagem atômica
                break;
            }
        }
    }
}

int main() {
    int num_notas = 10000000;
    int num_faixas = 10;

    float *notas_h = (float *)malloc(num_notas * sizeof(float));
    srand(42);
    for (int i = 0; i < num_notas; i++) {
        notas_h[i] = ((float) rand() / RAND_MAX) * 10.0;  // Gera valores entre 0 e 10
    }

    float faixas_h[] = {0.0, 1.0, 1.1, 2.0, 2.1, 3.0, 3.1, 4.0, 4.1, 5.0,
                        5.1, 6.0, 6.1, 7.0, 7.1, 8.0, 8.1, 9.0, 9.1, 10.0};

    float *soma_histograma_h = (float *)calloc(num_faixas, sizeof(float));
    int *contagem_histograma_h = (int *)calloc(num_faixas, sizeof(int));

    float *notas_d, *faixas_d, *soma_histograma_d;
    int *contagem_histograma_d;

    cudaMalloc((void **)&notas_d, num_notas * sizeof(float));
    cudaMalloc((void **)&faixas_d, 2 * num_faixas * sizeof(float));
    cudaMalloc((void **)&soma_histograma_d, num_faixas * sizeof(float));
    cudaMalloc((void **)&contagem_histograma_d, num_faixas * sizeof(int));

    cudaMemcpy(notas_d, notas_h, num_notas * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(faixas_d, faixas_h, 2 * num_faixas * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(soma_histograma_d, soma_histograma_h, num_faixas * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(contagem_histograma_d, contagem_histograma_h, num_faixas * sizeof(int), cudaMemcpyHostToDevice);

    int num_blocos = (num_notas + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start, 0);

    calcular_histograma<<<num_blocos, THREADS_PER_BLOCK>>>(notas_d, faixas_d, num_notas, num_faixas, soma_histograma_d, contagem_histograma_d);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    cudaMemcpy(soma_histograma_h, soma_histograma_d, num_faixas * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(contagem_histograma_h, contagem_histograma_d, num_faixas * sizeof(int), cudaMemcpyDeviceToHost);

    for (int i = 0; i < num_faixas; i++) {
        printf("Faixa %.1f a %.1f:\n", faixas_h[2 * i], faixas_h[2 * i + 1]);
        printf("  Soma dos valores: %.2f\n", soma_histograma_h[i]);
        printf("  Contagem de valores: %d\n", contagem_histograma_h[i]);
    }

    printf("Tempo de execução: %.4f segundos\n", milliseconds / 1000);

    cudaFree(notas_d);
    cudaFree(faixas_d);
    cudaFree(soma_histograma_d);
    cudaFree(contagem_histograma_d);
    free(notas_h);
    free(soma_histograma_h);
    free(contagem_histograma_h);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

Overwriting histograma_cuda.cu


In [14]:
!nvcc histograma_cuda.cu -o histograma_cuda


In [15]:
!./histograma_cuda


Faixa 0.0 a 1.0:
  Soma dos valores: 500527.16
  Contagem de valores: 1001164
Faixa 1.1 a 2.0:
  Soma dos valores: 1394522.12
  Contagem de valores: 899573
Faixa 2.1 a 3.0:
  Soma dos valores: 2296783.50
  Contagem de valores: 900287
Faixa 3.1 a 4.0:
  Soma dos valores: 3198549.00
  Contagem de valores: 900488
Faixa 4.1 a 5.0:
  Soma dos valores: 4096549.75
  Contagem de valores: 899855
Faixa 5.1 a 6.0:
  Soma dos valores: 4996083.50
  Contagem de valores: 899604
Faixa 6.1 a 7.0:
  Soma dos valores: 5895497.00
  Contagem de valores: 899519
Faixa 7.1 a 8.0:
  Soma dos valores: 6794154.50
  Contagem de valores: 899423
Faixa 8.1 a 9.0:
  Soma dos valores: 7691738.00
  Contagem de valores: 899229
Faixa 9.1 a 10.0:
  Soma dos valores: 8599928.00
  Contagem de valores: 900063
Tempo de execução: 0.0267 segundos
